Unfortunately Github doesn't render ipywidgets, which allow us to interactively alter the parameters of the model. If you want to mess with the parameters yourself then download this file, and also install and activate ipywidgets, which you can do in the command line interface with the following commands:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

In [1]:
#dependencies
import numpy as np
import operator as op
from functools import reduce
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

Parameters:
<br>
$\lambda$ = probability of encountering resources <br>
$S(t)$ = number of solitary locusts at time t [locusts] <br>
$G(t)$ = number of gregarious locusts at time t [locusts] <br>
$R(t)$ = number of resources at time t [mass] <br>
$R^*$ = Critical value of resources <br>
$\alpha$ = percentage of total resources found in a patch, proxy for clumpiness <br>
$k$ = proportion of resources reserved for a solitary locust before gregarious locusts arrive <br>
$h_0$ = foraging rate [1/time] <br>
$\mu$ = energy cost [mass/locust] <br> 
$r$ = resource regeneration rate [1/time] <br>
$p_s, p_g$ = proportion of solitary, gregarious locusts <br>
Note that $G(t)$ and $S(t)$ actually stay the same, but the energy of locusts' changes

<h3>The model:</h3><br>
<h4>Change in energy for gregarious locusts</h4><br>
$$
G(t)\frac{dE_G}{dt} = \left [\sum_{i=0}^{S(t)}{S(t) \choose i}\lambda_s^i(1-\lambda_s)^{S(t)-i} i\right ](1-p_sk) \alpha p_sh(R) + \lambda_g \alpha p_gh(R)G(t) - \mu_GG(t)
$$
Where $h(R)=h_0\frac{R(t)}{(R(t)+R^*)}$ (Instead of $R(t)$ this could be the expected number of resources)
<p>Gregarious locusts gain energy both by discovering resources themselves (second term on the right) and by scrounging off of solitary locusts (first term)</p>


<h4>Change in energy for solitary locusts</h4>
<br>
$$
S(t)\frac{dE_S}{dt} = \left [\sum_{i=0}^{S(t)} {S(t) \choose i} \lambda_s^i(1-\lambda_s)^{S(t)-i}i \right ]p_sk \alpha h(R) - \mu_SS(t)
$$

<h4>Change in resources</h4>
$$
\frac{dR}{dt} = -(\frac{dE_S}{dt}+\frac{dE_G}{dt})+r
$$

In [2]:
#a helper function to compute n choose k
def choose(n,r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [3]:
##implementation of dE_G/dt
def DE(S, L, R, alpha, k, G, h, us, ug, r, Rn):
    glamb = G/L
    slamb = S/L
    ps=S/(S+G)
    pg=G/(S+G)
    
    #dE_G/dt
    x=0
    for i in range(S):
        x += choose(S,i)*(slamb**i)*((1-slamb)**(S-i))*i
    h = h*R/(R+Rn)
    x=x*h*ps*(1-ps*k)*alpha
    x += glamb*G*alpha*pg*h - ug*G
    
    #dE_S/dt
    y = 0
    for i in range(S):
        y += choose(S,i)*((slamb**i)*((1-slamb)**(S-i))*i)
    y = y*ps*k*alpha*h - us*S
    
    #dR/dt
    z = -1*(x+y) + r
    return x, y, z

In [4]:
@interact(k=(.05,1,.1), h=(.05, .5, .05), alpha=(0,1,.05), us=(.001,.015, .001), ug=(.001,.015, .001))
def simulate(S=(10,50), L=100, G=(10,50), Rn=30, alpha=.3, k=.35, h=.05, us=.004, ug=.007, r=.2, tsteps=10000):
    dt = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [0,0,Rn]
    for i in range(tsteps):
        g_dot, s_dot, R_dot = DE(S, L, x[i][2], alpha, k, G, h, us, ug, r, Rn)
        x[i+1][0] = x[i][0]+G*g_dot*dt
        x[i+1][1] = x[i][1]+S*s_dot*dt
        x[i+1][2] = x[i][2]+R_dot*dt
    g = [y[0] for y in x]
    s = [y[1] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=30, description='S', max=50, min=10), IntSlider(value=100, description='…

Resources are green, energy of solitary locusts is red, and energy of gregarious locusts is blue

<h3> Non Dimensionalization </h3>
<br>
We would like to make the system of equations dimensionless so that we can deal with fewer parameters <br>
First note that the sums in the first two DEs evaluate to the expected value of a binomial distribution, which in this case would be $S(t) \lambda_s$ 
<br>
We set up a time scale $\tau = t/T$ for $T=1/\mu_s$. Thus $\frac{dE_S}{dt}=\frac{dE_S}{d\tau}\frac{d\tau}{dt}=\mu_s\frac{dE_S}{d\tau}$
<br>
Thus our DE for solitary locusts' energy becomes:
$$
\mu_s S(t) \frac{dE_S}{d\tau} = S(t)\lambda_s p_sk \alpha h(R) - \mu_sS(t) = S(t)\lambda_s k \alpha h_0 \frac{S(t)}{S(t)+G(t)} \frac{R(t)}{R(t)+R^*} - \mu_sS(t) 
$$
<br>
Dividing through by $\mu_s S(t)$ yields
$$
\frac{dE_S}{d\tau} = S(t) k \gamma p_s \frac{R(t)}{R(t)+R^*} - 1 
$$

where $\gamma = \frac{\alpha h_0}{\mu_s L}$
<br>
<br>
We also apply this non dimensionalization to gregarious locusts' energy
$$
\mu_sG(t)\frac{dE_G}{d\tau} = S(t)\lambda_s\alpha p_s h(R) - S(t)\lambda_s\alpha kp_s^2h(R) + \lambda_g \alpha p_gh(R)G(t) - \mu_GG(t)
$$
Dividing through yields:
$$
\frac{dE_G}{d\tau} = S(t)q(1-kp_s)\gamma p_s\frac{R(t)}{R(t)+R^*} + G(t)\gamma p_g \frac{R(t)}{R(t)+R^*} - \mu_l
$$
where $\mu_l = \mu_g/\mu_s$ and $q=S(t)/G(t)$
<br>
<br>
Finally, non dimensionalizing the resource DE yields:
$$
\frac{dR}{d\tau} = -(\frac{dE_s}{d\tau} + \frac{dE_g}{d\tau}) + r^*
$$
where $r^*=r/\mu_s$

In [5]:
def altDE(S, R, k, G, ul, r, Rn, gamma):
    q = S/G
    ps = S/(S+G)
    pg = G/(S+G)
    h = R/(R+Rn)
    
    #dE_G/dtau
    x = S*q*(1-k*ps)*gamma*ps*h + G*gamma*pg*h - ul
    
    #dE_S/dt
    y = S*k*gamma*ps*h - 1
    
    #dR/dt
    z = -1*(x+y) + r
    return x, y, z

In [8]:
@interact(k=(.05,1,.1), S=(0,50,5), G=(0,50,5), ul=(1,3,.1), gamma=(.01,1,.01))
def simulate(S=15, G=15, Rn=30, k=.35, ul=1.5, r=.2, gamma=.008, tsteps=10000):
    dt = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [0,0,Rn]
    for i in range(tsteps):
        g_dot, s_dot, R_dot = altDE(S, x[i][2], k, G, ul, r, Rn, gamma)
        x[i+1][0] = x[i][0]+G*g_dot*dt
        x[i+1][1] = x[i][1]+S*s_dot*dt
        x[i+1][2] = x[i][2]+R_dot*dt
    g = [y[0] for y in x]
    s = [y[1] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=15, description='S', max=50, step=5), IntSlider(value=15, description='G…